In [1]:
#1. RUMUS TRIGONOMETRI
import tkinter as tk
import math

def update_triangle(angle_degrees):
    canvas.delete("all")
    
    angle = math.radians(float(angle_degrees))
    hypotenuse = 300  # Panjang sisi miring dalam pixel
    
    # Hitung sisi segitiga
    adjacent = hypotenuse * math.cos(angle)  # Sisi alas
    opposite = hypotenuse * math.sin(angle)  # Sisi tegak
    
    # ======== GAMBAR SEGITIGA ========
    # Alas (biru)
    canvas.create_line(100, 350, 100 + adjacent, 350, fill="blue", width=3)
    # Tinggi (hijau)
    canvas.create_line(100 + adjacent, 350, 100 + adjacent, 350 - opposite, fill="green", width=3)
    # Miring (merah)
    canvas.create_line(100, 350, 100 + adjacent, 350 - opposite, fill="red", width=3)
    
    # Gambar sudut
    canvas.create_arc(80, 330, 120, 370, start=0, extent=angle_degrees, 
                     outline="orange", width=2, style="arc")
    
    # ======== TEOREMA PYTHAGORAS ========
    pythagoras_text = (
        "TEOREMA PYTHAGORAS:\n"
        "Miring² = Alas² + Tinggi²\n"
        f"{hypotenuse}² = {adjacent:.1f}² + {opposite:.1f}²\n"
        f"{hypotenuse**2} = {adjacent**2:.1f} + {opposite**2:.1f}\n"
        f"{hypotenuse**2} ≈ {adjacent**2 + opposite**2:.1f}"
    )
    
    # ======== RUMUS TRIGONOMETRI ========
    trigono_text = (
        "\nRUMUS TRIGONOMETRI:\n"
        f"sin({angle_degrees}°) = Tinggi/Miring = {opposite:.1f}/{hypotenuse} = {math.sin(angle):.3f}\n"
        f"cos({angle_degrees}°) = Alas/Miring = {adjacent:.1f}/{hypotenuse} = {math.cos(angle):.3f}\n"
        f"tan({angle_degrees}°) = Tinggi/Alas = {opposite:.1f}/{adjacent:.1f} = {math.tan(angle):.3f}"
    )
    
    # ======== CONTOH KASUS NYATA ========
    examples_text = (
        "\nCONTOH APLIKASI:\n"
        "1. Tinggi dinding yang bisa di jangkau tangga (hijau):\n"
        f"   diketahui : panjang tangga (merah) dan sudut {angle_degrees}°\n"
        f"   Tinggi dinding = Miring(panjang tangga) × sinθ\n"
        f"   {opposite:.1f} = {hypotenuse} × {math.sin(angle):.3f}\n\n"
        "2. Menentukan panjang tangga(merah):\n"
        f"   diketahui : jarak alas tangga ke dinding dan sudut {angle_degrees}°\n"
        f"   Miring(panjang tangga) = Alas(jarak dinding) / cosθ\n"
        f"   {hypotenuse:.1f} = {adjacent:.1f} / {math.cos(angle):.3f}\n\n"
        "3. Mengukur tinggi pohon:\n"
        f"   diketahui : jarak ke pohon dan sudut {angle_degrees}°\n"
        f"   Tinggi = Alas(jarak ke pohon) × tanθ\n"
        f"   {opposite:.1f} = {adjacent:.1f} × {math.tan(angle):.3f}"
    )
    
    # Gabungkan semua teks
    info_text = pythagoras_text + trigono_text + examples_text
    
    # Tampilkan teks
    canvas.create_text(400, 50, text=info_text, font=("Arial", 11), 
                      anchor="nw", justify=tk.LEFT)
    
    # Label sisi-sisi segitiga
    canvas.create_text(100, 320, text=f"Sudut: {angle_degrees}°", 
                      font=("Arial", 10, "bold"))
    canvas.create_text(100 + adjacent/2, 370, text=f"Alas: {adjacent:.1f} px", 
                      font=("Arial", 10, "bold"))
    canvas.create_text(100 + adjacent + 20, 350 - opposite/2, 
                      text=f"Tinggi: {opposite:.1f} px", font=("Arial", 10, "bold"))
    canvas.create_text(100 + adjacent/2 - 30, 350 - opposite/2 - 20, 
                      text=f"Miring: {hypotenuse} px", font=("Arial", 10, "bold"))

    # Lingkaran (untuk visualisasi sudut)
    center_x, center_y = 100, 350
    radius = 300
    canvas.create_oval(center_x - radius, center_y - radius, 
                    center_x + radius, center_y + radius, outline="blue")

# Membuat window
root = tk.Tk()
root.title("Pythagoras & Trigonometri dalam Kehidupan Sehari-hari")
canvas = tk.Canvas(root, width=750, height=550, bg="white")
canvas.pack()

# Slider untuk sudut
angle_slider = tk.Scale(root, from_=1, to=89, resolution=1, orient=tk.HORIZONTAL,
                       label="Atur sudut θ (derajat):",
                       command=update_triangle)
angle_slider.set(30)
angle_slider.pack(fill=tk.X, padx=20, pady=10)

update_triangle(30)
root.mainloop()


In [50]:
#Simulasi Visual Pythagoras pada Segitiga Siku-Siku > COPY PASTE
import tkinter as tk
import math

def update_canvas(angle_deg):
    canvas.delete("all")
    cx, cy = 250, 250
    r = 150
    theta = math.radians(angle_deg)

    x_cos = r * math.cos(theta)
    y_sin = r * math.sin(theta)

    # Titik ujung garis horizontal
    hx = cx + x_cos
    hy = cy

    # Titik ujung garis vertikal
    px = hx
    py = hy - y_sin

    # Gambar garis horizontal (r*cos(θ))
    canvas.create_line(cx, cy, hx, hy, fill='orange', width=3)
    canvas.create_text((cx+hx)/2, hy+15, text=f"x=r*cos(θ) = {x_cos:.2f}", fill='orange', font=("Arial", 12))

    # Gambar garis vertikal (r*sin(θ))
    canvas.create_line(hx, hy, px, py, fill='blue', width=3)
    canvas.create_text(hx+60, (hy+py)/2, text=f"y=r*sin(θ) = {y_sin:.2f}", fill='blue', font=("Arial", 12))

    # Gambar sisi miring (r)
    canvas.create_line(px, py, cx, cy, fill='green', width=3)
    canvas.create_text((cx+px)/2 - 20, (cy+py)/2, text=f"r = {r}", fill='green', font=("Arial", 12))

    # Gambar titik-titik
    canvas.create_oval(cx-4, cy-4, cx+4, cy+4, fill='black')  # Pusat
    canvas.create_oval(px-4, py-4, px+4, py+4, fill='red')    # Titik ujung
    canvas.create_text(px+10, py-8, anchor='nw', text=f"({px-cx:.2f},{py-cy:.2f})", font=("Arial", 11))
    canvas.create_text(hx+10, hy-8, anchor='nw', text=f"({hx-cx:.2f},{hy-cy:.2f})", font=("Arial", 11))

    # Tulisan sudut
    canvas.create_text(10, 10, anchor='nw', text=f"θ = {angle_deg}°", font=("Arial", 14))
    canvas.create_text(cx-50, cy-10, anchor='nw', text=f"θ = {angle_deg}°", font=("Arial", 11))

    # Pythagoras check
    lhs = r ** 2
    rhs = x_cos ** 2 + y_sin ** 2
    check_text = f"Check: r² = (r*cos(θ))² + (r*sin(θ))²\n{r}² = {x_cos:.2f}² + {y_sin:.2f}²\n{lhs:.2f} = {rhs:.2f}"
    canvas.create_text(10, 40, anchor='nw', text=check_text, font=("Courier", 12), fill='black')
    check_text1 = f"dot black center: {cx, cy} >> (0,0)\ndot red ({px:.2f},{py:.2f})>>({px-cx:.2f},{py-cy:.2f}) "
    canvas.create_text(10, 400, anchor='nw', text=check_text1, font=("Arial", 12), fill='black')

    canvas.create_oval(cx-r, cy-r, cx+r, cy+r, outline="red")
    
def on_slider_change(value):
    angle_deg = int(value)
    update_canvas(angle_deg)

root = tk.Tk()
root.title("Simulasi Visual Pythagoras pada Segitiga Siku-Siku")

canvas = tk.Canvas(root, width=500, height=500, bg='white')
canvas.pack()

slider = tk.Scale(root, from_=0, to=360, orient='horizontal', label='Atur θ (derajat)', command=on_slider_change)
slider.pack()
slider.set(45)  # Default sudut 45°

update_canvas(45)

root.mainloop()


In [ ]:
#Pemahaman Segitiga Sama Sisi: Visual dan Rumus COPY PASTE
import tkinter as tk
import math

def get_triangle_coords(center, r, angles):
    x, y = center
    points = []
    for theta in angles:
        px = x + r * math.cos(theta)
        py = y + r * math.sin(theta)
        points.append((px, py))
    return points

root = tk.Tk()
root.title("Pemahaman Segitiga Sama Sisi: Visual dan Rumus")

canvas = tk.Canvas(root, width=500, height=500, bg='white')
canvas.pack()

center = (200, 200)
r = 100
angles = [0, 2*math.pi/3, 4*math.pi/3]  # 0°, 120°, 240°

points = get_triangle_coords(center, r, angles)

# Menggambar segitiga
canvas.create_polygon([p for point in points for p in point], fill='lightblue', outline='black', width=2)

# Titik pusat
cx, cy = center
canvas.create_oval(cx-4, cy-4, cx+4, cy+4, fill='green')
canvas.create_text(cx+10, cy, text=f"Center ({int(cx)}, {int(cy)})", anchor='w', font=("Arial", 10))

canvas.create_oval(cx-r, cy-r, cx+r, cy+r, outline="red")

# Tampilkan titik-titik sudut dengan hasil perhitungan
for idx, (px, py) in enumerate(points, start=1):
    canvas.create_oval(px-4, py-4, px+4, py+4, fill='red')
    canvas.create_text(px+10, py, text=f"P{idx} ({int(px)}, {int(py)})", anchor='w', font=("Arial", 10))

# Tampilkan rumus di layar
rumus = [
    "Rumus Mencari Titik Sudut Segitiga:",
    "px = x + r * cos(theta)",
    "py = y + r * sin(theta)",
    "",
    f"x = {center[0]}, y = {center[1]}, r = {r}",
    "theta = 0°, 120°, 240° (0, 2.094, 4.188 rad)",
    "",
    f"Hasil:",
    f"P1: ({int(points[0][0])}, {int(points[0][1])})",
    f"P2: ({int(points[1][0])}, {int(points[1][1])})",
    f"P3: ({int(points[2][0])}, {int(points[2][1])})"
]

for i, line in enumerate(rumus):
    canvas.create_text(10, 10 + i*18, text=line, anchor='nw', font=("Courier", 10), fill='black')

root.mainloop()


In [ ]:
#Segitiga Sama Kaki
import tkinter as tk
import math

root = tk.Tk()
root.title("Segitiga Sama Kaki + Garis Bantu Sin/Cos")

canvas = tk.Canvas(root, width=500, height=700, bg='white')
canvas.pack()

# Parameter segitiga
center = (200, 200)
r = 120
theta = math.radians(30)  # sudut kaki terhadap vertikal (30°)

cx, cy = center

# Hitung titik-titik segitiga sama kaki
p1 = (cx - r * math.sin(theta), cy - r * math.cos(theta))  # kiri atas
p2 = (cx + r * math.sin(theta), cy - r * math.cos(theta))  # kanan atas
p3 = (cx, cy + r)                                          # bawah

# Gambar segitiga
canvas.create_polygon([p1, p2, p3], fill='lightblue', outline='black', width=2)

# Titik pusat
canvas.create_oval(cx-4, cy-4, cx+4, cy+4, fill='green')
canvas.create_text(cx+10, cy, text=f"Center ({int(cx)}, {int(cy)})", anchor='w', font=("Arial", 10))

# Gambar titik-titik sudut
for idx, (px, py) in enumerate([p1, p2, p3], start=1):
    canvas.create_oval(px-4, py-4, px+4, py+4, fill='red')
    canvas.create_text(px+10, py, text=f"P{idx} ({int(px)}, {int(py)})", anchor='w', font=("Arial", 10))

# Garis bantu untuk P1 (kiri atas)
hx1 = cx - r * math.sin(theta)  # horizontal ke kiri
hy1 = cy                         # tetap di pusat y
vx1 = hx1
vy1 = cy - r * math.cos(theta)   # naik ke atas

canvas.create_line(cx, cy, hx1, hy1, fill='orange', dash=(4,2), width=2)  # r * sin(θ)
canvas.create_line(hx1, hy1, vx1, vy1, fill='blue', dash=(4,2), width=2)  # r * cos(θ)

canvas.create_text((cx+hx1)//2, hy1 - 10, text="r * sin(θ)", fill='orange', font=("Arial", 10))
canvas.create_text(hx1 + 10, (hy1+vy1)//2, text="r * cos(θ)", fill='blue', font=("Arial", 10))

# Garis bantu untuk P2 (kanan atas)
hx2 = cx + r * math.sin(theta)  # horizontal ke kanan
hy2 = cy
vx2 = hx2
vy2 = cy - r * math.cos(theta)

canvas.create_line(cx, cy, hx2, hy2, fill='orange', dash=(4,2), width=2)  # r * sin(θ)
canvas.create_line(hx2, hy2, vx2, vy2, fill='blue', dash=(4,2), width=2)  # r * cos(θ)

canvas.create_text((cx+hx2)//2, hy2 - 10, text="r * sin(θ)", fill='orange', font=("Arial", 10))
canvas.create_text(hx2 + 10, (hy2+vy2)//2, text="r * cos(θ)", fill='blue', font=("Arial", 10))

canvas.create_oval(cx-r, cy-r, cx+r, cy+r, outline="red")

# Penjelasan teks di layar
penjelasan = [
    "Segitiga Sama Kaki dengan Garis Bantu Sin/Cos",
    f"r = {r}",
    "P1 Titik atas kiri diperoleh dengan:",
    "  px = x - r * sin(theta)",
    f" {vx1} = {cx} - {r} * math.sin({theta:.3})",
    "  py = y - r * cos(theta)",
    f" {vy1:.3} = {cy} - {r} * math.cos({theta:.3})",
    "",
    "P2 Titik atas kanan diperoleh dengan:",
    "  px = x + r * sin(theta)",
    f" {vx2} = {cx} + {r} * math.sin({theta:.3})",
    "  py = y - r * cos(theta)",
    f" {vy2:.3} = {cy} - {r} * math.cos({theta:.3})",
    "",
    "P3 Titik bawah tengah diperoleh dengan:",
    f"  px = x = {cx}",
    f"  py = y + r ={cy + r}",
    "",
    "Garis orange: r * sin(theta) (horizontal)",
    "Garis biru: r * cos(theta) (vertikal)"
]

for i, line in enumerate(penjelasan):
    canvas.create_text(10, 330 + i*18, text=line, anchor='nw', font=("Courier", 10), fill='black')

root.mainloop()


In [ ]:
#Vehicle Bergerak
import tkinter as tk
import math
from vector import Vector

class Vehicle:
    def __init__(self, x, y, canvas):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.velocity = Vector(1, 0.5)  # bergerak pelan ke kanan dan bawah
        self.r = 6  # ukuran
        self.shape = self.canvas.create_polygon(0, 0, 0, 0, 0, 0, fill='lightblue', outline='black', width=2)
        self.update()

    def update(self):
        # Update posisi vehicle
        self.position.add(self.velocity)
        self.show()
        self.canvas.after(30, self.update)  # panggil update setiap 30 ms

    def show(self):
        angle = self.velocity.heading()  # arah segitiga mengikuti arah velocity

        # Titik depan, kiri, kanan
        front = Vector(math.cos(angle), math.sin(angle)).multed(self.r * 20)
        left  = Vector(math.cos(angle + 2.5), math.sin(angle + 2.5)).multed(self.r * 20)
        right = Vector(math.cos(angle - 2.5), math.sin(angle - 2.5)).multed(self.r * 20)

        # Tambahkan posisi pusat vehicle
        p1 = self.position.added(front)
        p2 = self.position.added(left)
        p3 = self.position.added(right)

        # Gambar segitiga
        self.canvas.coords(self.shape, p1.x, p1.y, p2.x, p2.y, p3.x, p3.y)

        # Hapus label sebelumnya
        self.canvas.delete('info')

        # Label titik-titik untuk belajar
        for p, label in zip([p1, p2, p3], ['P1', 'P2', 'P3']):
            self.canvas.create_oval(p.x-3, p.y-3, p.x+3, p.y+3, fill='red', tags='info')
            self.canvas.create_text(p.x+10, p.y, text=f"{label} ({int(p.x)}, {int(p.y)})", anchor='w', font=("Arial", 9), tags='info')

        # Titik pusat
        cx, cy = self.position.x, self.position.y
        self.canvas.create_oval(cx-3, cy-3, cx+3, cy+3, fill='green', tags='info')
        self.canvas.create_text(cx+10, cy, text=f"Center ({int(cx)}, {int(cy)})", anchor='w', font=("Arial", 9), tags='info')

# Membuat window
root = tk.Tk()
root.title("Vehicle Bergerak ")

# Membuat canvas
canvas = tk.Canvas(root, width=500, height=400, bg='white')
canvas.pack()

# Membuat Vehicle pada posisi awal (100, 100)
v = Vehicle(100, 100, canvas)

root.mainloop()


In [ ]:
#COPY PASTE
import tkinter as tk
import math
from vector import Vector 

class Vehicle:
    def __init__(self, x, y, canvas):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.velocity = Vector(1, 0.5)
        self.max_speed = 5
        self.max_force = 0.1
        self.r = 6  # Ukuran dasar
        self.color = "lightblue"
        
        # Buat shape segitiga
        self.shape = self.canvas.create_polygon(0, 0, 0, 0, 0, 0, 
                                              fill=self.color, 
                                              outline='black', 
                                              width=2)
        self.debug_text = self.canvas.create_text(0, 0, text="", 
                                                font=("Arial", 8),
                                                anchor='nw')
        self.update()

    def apply_force(self, force):
        """Terapkan gaya dengan limit max_force"""
        force.limit(self.max_force)
        self.velocity.add(force)
        self.velocity.limit(self.max_speed)

    def update(self):
        """Update posisi dan orientasi"""
        # 1. Update posisi
        self.position.add(self.velocity)
        
        # 2. Handle boundary (wrap-around)
        self.wrap_around()
        
        # 3. Update tampilan
        self.show()
        
        # 4. Panggil update berikutnya
        self.canvas.after(30, self.update)

    def wrap_around(self):
        """Buat vehicle muncul di sisi lain jika keluar canvas"""
        if self.position.x > 500:
            self.position.x = 0
        elif self.position.x < 0:
            self.position.x = 500
            
        if self.position.y > 400:
            self.position.y = 0
        elif self.position.y < 0:
            self.position.y = 400

    def show(self):
        """Gambar vehicle menghadap arah velocity"""
        # 1. Hitung arah hadap
        if self.velocity.magnitude() > 0:
            heading = self.velocity.normalized()
        else:
            heading = Vector(1, 0)  # Default jika velocity nol
        
        # 2. Hitung 3 titik segitiga
        front = heading.multed(self.r * 3)  # Titik depan (lebih panjang)
        left = Vector(-heading.y, heading.x).multed(self.r)  # Kiri belakang
        right = Vector(heading.y, -heading.x).multed(self.r)  # Kanan belakang
        
        # 3. Posisi aktual di canvas
        p1 = self.position.added(front)
        p2 = self.position.added(left)
        p3 = self.position.added(right)
        
        # 4. Update bentuk segitiga
        self.canvas.coords(self.shape, 
                          p1.x, p1.y, 
                          p2.x, p2.y, 
                          p3.x, p3.y)
        
        # 5. Update debug info
        self.update_debug_info(p1, p2, p3)

    def update_debug_info(self, p1, p2, p3):
        """Tampilkan informasi debug"""
        debug_text = (
            f"Position: ({self.position.x:.1f}, {self.position.y:.1f})\n"
            f"Velocity: ({self.velocity.x:.2f}, {self.velocity.y:.2f})\n"
            f"Speed: {self.velocity.magnitude():.2f}\n"
            f"Angle: {math.degrees(self.velocity.heading()):.1f}°"
        )
        self.canvas.coords(self.debug_text, 
                          self.position.x + 15, 
                          self.position.y - 30)
        self.canvas.itemconfig(self.debug_text, text=debug_text)

class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Advanced Vehicle Simulation")
        
        # Setup canvas
        self.canvas = tk.Canvas(root, width=500, height=400, bg='white')
        self.canvas.pack()
        
        # Kontrol keyboard
        self.root.bind('<KeyPress>', self.handle_keypress)
        
        # Buat vehicle
        self.vehicle = Vehicle(100, 100, self.canvas)
        
        # Label instruksi
        self.instructions = tk.Label(root, 
                                   text="Gunakan WASD untuk mengontrol", 
                                   font=("Arial", 10))
        self.instructions.pack()

    def handle_keypress(self, event):
        """Tangani input keyboard untuk kontrol manual"""
        force = Vector(0, 0)
        
        if event.keysym.lower() == 'w':
            force.y = -0.5  # Maju
        elif event.keysym.lower() == 's':
            force.y = 0.5   # Mundur
        elif event.keysym.lower() == 'a':
            force.x = -0.5  # Kiri
        elif event.keysym.lower() == 'd':
            force.x = 0.5   # Kanan
            
        self.vehicle.apply_force(force)

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()

In [ ]:
#Vehicle Bergerak + Berputar Konstan
import tkinter as tk
import math
from vector import Vector

class App:
    def __init__(self, width=500, height=400):
        self.root = tk.Tk()
        self.root.title("Vehicle Bergerak + Berputar Konstan")

        self.canvas = tk.Canvas(self.root, width=width, height=height, bg='white')
        self.canvas.pack()

        self.vehicle = Vehicle(100, 100, self.canvas)

        self.root.mainloop()

class Vehicle:
    def __init__(self, x, y, canvas):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.velocity = Vector(1, 0.5)  # bergerak ke kanan & bawah
        self.angle = 0  # sudut awal untuk rotasi independen
        self.r = 6  # ukuran
        self.shape = self.canvas.create_polygon(0, 0, 0, 0, 0, 0, fill='lightblue', outline='black', width=2)
        self.update()

    def update(self):
        self.position.add(self.velocity)
        self.angle += 0.002  # radian per frame, rotasi konstan
        self.show()
        self.canvas.after(60, self.update)

    def show(self):
        angle = self.angle

        front = Vector(math.cos(angle), math.sin(angle)).multed(self.r * 20)
        left = Vector(math.cos(angle + 2.5), math.sin(angle + 2.5)).multed(self.r * 20)
        right = Vector(math.cos(angle - 2.5), math.sin(angle - 2.5)).multed(self.r * 20)

        p1 = self.position.added(front)
        p2 = self.position.added(left)
        p3 = self.position.added(right)

        self.canvas.coords(self.shape, p1.x, p1.y, p2.x, p2.y, p3.x, p3.y)

        self.canvas.delete('info')

        for p, label in zip([p1, p2, p3], ['P1', 'P2', 'P3']):
            self.canvas.create_oval(p.x - 3, p.y - 3, p.x + 3, p.y + 3, fill='red', tags='info')
            self.canvas.create_text(p.x + 10, p.y, text=f"{label} ({int(p.x)}, {int(p.y)})", anchor='w', font=("Arial", 9), tags='info')

        cx, cy = self.position.x, self.position.y
        self.canvas.create_oval(cx - 3, cy - 3, cx + 3, cy + 3, fill='green', tags='info')
        self.canvas.create_text(cx + 10, cy, text=f"Center ({int(cx)}, {int(cy)})", anchor='w', font=("Arial", 9), tags='info')

if __name__ == "__main__":
    App()

In [ ]:
import math

# Derajat ke radian
deg = 60
rad = math.radians(deg)
print(f"{deg}° = {rad:.2f} rad") #60° = 1.05 rad

# Radian ke derajat
rad2 = math.pi / 3
deg2 = math.degrees(rad2)
print(f"{rad2:.2f} rad = {deg2:.1f}°") #1.05 rad = 60.0°

60° = 1.05 rad
1.05 rad = 60.0°


In [ ]:
#Simulasi Rotasi Sederhana - Angular Velocity Konstan
import tkinter as tk
import math
from vector import Vector

class App:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("🔄 Simulasi Rotasi Sederhana - Angular Velocity Konstan")

        self.canvas = tk.Canvas(self.root, width=600, height=400, bg="white")
        self.canvas.pack()

        # Pusat rotasi
        self.center = Vector(300, 200)
        self.radius = 100
        self.angle_deg = 0
        self.angular_velocity = 3  # derajat per frame

        # Objek visual
        self.linex = self.canvas.create_line(200, 200, 400, 200, fill="green", width=2)
        self.liney = self.canvas.create_line(300, 100, 300, 300, fill="gray", width=2)
        self.circle = self.canvas.create_oval(self.center.x - 5, self.center.y - 5, self.center.x + 5, self.center.y + 5, fill="black")
        self.dot = self.canvas.create_oval(0, 0, 0, 0, fill="red")
        self.line = self.canvas.create_line(0, 0, 0, 0, fill="blue", width=2)
        self.text = self.canvas.create_text(10, 10, anchor="nw", text="", font=("Arial", 12), fill="black")

        self.update()
        self.root.mainloop()

    def update(self):
        angle_rad = math.radians(self.angle_deg)
        offset = Vector(math.cos(angle_rad), math.sin(angle_rad)).multed(self.radius)
        point = self.center.added(offset)

        # Gambar ulang titik dan garis
        self.canvas.coords(self.dot, point.x - 10, point.y - 10, point.x + 10, point.y + 10)
        self.canvas.coords(self.line, self.center.x, self.center.y, point.x, point.y)

        # Informasi teks
        info = f"""
🌀 ROTASI DENGAN KECEPATAN SUDUT KONSTAN

Sudut: {self.angle_deg:.1f}°
Sudut (radian): {angle_rad:.2f}
Kecepatan sudut (ω): {self.angular_velocity}°/frame
{offset} + {self.center} = {point}
x = cx + r × cos(θ) {offset}
  = {self.center.x} + {self.radius} × cos({self.angle_deg:.1f}°) = {point.x:.1f}
y = cy + r × sin(θ)
  = {self.center.y} + {self.radius} × sin({self.angle_deg:.1f}°) = {point.y:.1f}
"""
        self.canvas.itemconfig(self.text, text=info)

        # Tambah sudut
        self.angle_deg = (self.angle_deg + self.angular_velocity) % 360

        self.root.after(100, self.update)

if __name__ == "__main__":
    App()


In [25]:
#SCRIpT 1 : Simulasi Rotasi Baton, angular velocity constan
import tkinter as tk
import math
from vector import Vector 

# Class Baton (tongkat)
class Baton:
    def __init__(self, canvas, center, length):
        self.canvas = canvas
        self.center = center                # Vector posisi tengah
        self.length = length                # Panjang tongkat
        self.angle_deg = 0                  # Sudut awal (dalam derajat)
        self.angular_velocity = 2           # Kecepatan sudut (derajat/frame)
        self.angular_acceleration = 0       # Percepatan sudut (tetap nol)
        
        # Garis tongkat di canvas
        self.line = canvas.create_line(0, 0, 0, 0, width=8, fill="purple")
        self.linex = self.canvas.create_line(200, 200, 400, 200, fill="green", width=2)
        self.liney = self.canvas.create_line(300, 100, 300, 300, fill="gray", width=2)
        self.circle = self.canvas.create_oval(self.center.x - 5, self.center.y - 5, self.center.x + 5, self.center.y + 5, fill="black")
        self.dot1 = self.canvas.create_oval(0, 0, 0, 0, fill="red")
        self.dot2 = self.canvas.create_oval(0, 0, 0, 0, fill="blue")
        # Teks informasi di sudut layar
        self.text = canvas.create_text(10, 10, anchor="nw", text="", font=("Arial", 12), fill="black")

    def update(self):
        # Konversi sudut ke radian
        angle_rad = math.radians(self.angle_deg)

        # Buat arah unit vector berdasarkan sudut
        direction = Vector(math.cos(angle_rad), math.sin(angle_rad))

        # Hitung dua ujung tongkat berdasarkan pusat
        end1 = self.center.added(direction.multed(self.length))
        end2 = self.center.subbed(direction.multed(self.length))

        self.canvas.coords(self.dot1, end1.x - 10, end1.y - 10, end1.x + 10, end1.y + 10)
        self.canvas.coords(self.dot2, end2.x - 10, end2.y - 10, end2.x + 10, end2.y + 10)

        # Perbarui garis tongkat pada canvas
        self.canvas.coords(self.line, end1.x, end1.y, end2.x, end2.y)

        # Tampilkan informasi perhitungan pada layar
        info = f"""Sudut (derajat) red : {self.angle_deg:.1f}°
Sudut (radian): {angle_rad:.2f}
Angular Velocity: {self.angular_velocity:.2f}°/frame
Angular Acceleration: {self.angular_acceleration:.2f}°/frame²
Ujung 1 red: ({end1.x:.1f}, {end1.y:.1f})
Ujung 2 blue: ({end2.x:.1f}, {end2.y:.1f})
"""
        self.canvas.itemconfig(self.text, text=info)

        # Update sudut dengan angular velocity
        self.angle_deg = (self.angle_deg + self.angular_velocity) % 360

# Inisialisasi Tkinter dan kanvas
root = tk.Tk()
root.title("Simulasi Rotasi Baton - Nature of Code 3.1")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Titik tengah canvas sebagai Vector
center = Vector(300, 200)

# Buat objek Baton
baton = Baton(canvas, center, length=100)

# Fungsi animasi
def animate():
    baton.update()
    root.after(100, animate)

animate()
root.mainloop()


In [42]:
import tkinter as tk
import math
from vector import Vector

class Mover:
    def __init__(self, canvas, center, radius):
        self.canvas = canvas
        self.center = center
        self.radius = radius
        self.angle_rad = 0
        self.angular_velocity = 0.03
        self.angular_acceleration = 0.0001

        self.dot = self.canvas.create_oval(0, 0, 0, 0, fill="red")
        self.line = self.canvas.create_line(0, 0, 0, 0, fill="blue", width=2)
        self.text = self.canvas.create_text(10, 10, anchor="nw", text="", font=("Arial", 12), fill="black")

        self.update()

    def update(self):
        direction = Vector(math.cos(self.angle_rad), math.sin(self.angle_rad))
        point = self.center.added(direction.multed(self.radius))

        self.canvas.coords(self.dot, point.x - 10, point.y - 10, point.x + 10, point.y + 10)
        self.canvas.coords(self.line, self.center.x, self.center.y, point.x, point.y)

        info = f"""
🔁 ROTASI DENGAN PERCEPATAN SUDUT

Sudut (radian): {self.angle_rad:.4f}
Sudut (derajat): {math.degrees(self.angle_rad):.1f}°
Angular Velocity (ω): {self.angular_velocity:.5f} rad/frame
Angular Acceleration (α): {self.angular_acceleration:.5f} rad/frame²

x = cx + r × cos(θ) = {self.center.x} + {self.radius} × {direction.x:.3f} = {point.x:.1f}
y = cy + r × sin(θ) = {self.center.y} + {self.radius} × {direction.y:.3f} = {point.y:.1f}
"""
        self.canvas.itemconfig(self.text, text=info)

        self.angular_velocity += self.angular_acceleration
        self.angle_rad = (self.angle_rad + self.angular_velocity) % (2 * math.pi)

        self.canvas.after(20, self.update)

class App:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("🔁 Rotasi dengan Percepatan Sudut")

        self.canvas = tk.Canvas(self.root, width=600, height=400, bg="white")
        self.canvas.pack()

        center = Vector(300, 200)
        radius = 100

        # Garis referensi
        self.canvas.create_line(200, 200, 400, 200, fill="green", width=2)
        self.canvas.create_line(300, 100, 300, 300, fill="gray", width=2)
        self.canvas.create_oval(center.x - 5, center.y - 5, center.x + 5, center.y + 5, fill="black")

        self.mover = Mover(self.canvas, center, radius)

        self.root.mainloop()

if __name__ == "__main__":
    App()


In [43]:
#SCRIPT 2 : Simulasi Rotasi Baton, angular_velocity = 0.05, angular_acceleration = 0.001 
import tkinter as tk
import math
from vector import Vector

# ==========================
# Class Baton (Tongkat)
# ==========================
class Baton:
    def __init__(self, center, length):
        self.center = center                # Titik pusat rotasi (Vector)
        self.length = length               # Panjang tongkat

        self.angle = 0                     # Sudut awal (radian)
        self.angular_velocity = 0.05       # Kecepatan sudut
        self.angular_acceleration = 0.0005 # Percepatan sudut

        self.line = None                   # ID garis pada canvas
        self.circle1 = None                # Lingkaran ujung kiri
        self.circle2 = None                # Lingkaran ujung kanan

    def update(self):
        # Update sudut dan kecepatan sudut
        self.angle += self.angular_velocity
        self.angular_velocity += self.angular_acceleration

    def display(self, canvas):
        # Hitung arah rotasi sebagai unit vector
        direction = Vector(math.cos(self.angle), math.sin(self.angle))
        half = direction.multed(self.length / 2)

        # Ujung kiri dan kanan dari pusat
        end1 = self.center.subbed(half)
        end2 = self.center.added(half)

        if self.line is None:
            # Pertama kali buat objek canvas
            self.line = canvas.create_line(end1.x, end1.y, end2.x, end2.y, width=4, fill="blue")
            self.circle1 = canvas.create_oval(end1.x-6, end1.y-6, end1.x+6, end1.y+6, fill="red")
            self.circle2 = canvas.create_oval(end2.x-6, end2.y-6, end2.x+6, end2.y+6, fill="red")
        else:
            # Update posisi objek
            canvas.coords(self.line, end1.x, end1.y, end2.x, end2.y)
            canvas.coords(self.circle1, end1.x-6, end1.y-6, end1.x+6, end1.y+6)
            canvas.coords(self.circle2, end2.x-6, end2.y-6, end2.x+6, end2.y+6)

        return self.angle, self.angular_velocity, self.angular_acceleration

# ==========================
# Fungsi Update Layar
# ==========================
def update():
    baton.update()
    angle, a_vel, a_acc = baton.display(canvas)

    # Tampilkan informasi di layar
    info_text = (
        f"Angle (rad) = {angle:.3f}\n"
        f"Angular Velocity = {a_vel:.3f}\n"
        f"Angular Acceleration = {a_acc:.5f}"
    )
    label_info.config(text=info_text)

    window.after(30, update)  # Loop animasi

# ==========================
# Setup Tkinter Window
# ==========================
WIDTH, HEIGHT = 600, 400
window = tk.Tk()
window.title("Simulasi Rotasi Sudut - The Nature of Code 3.2")

canvas = tk.Canvas(window, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

label_info = tk.Label(window, text="", font=("Arial", 12), justify="left")
label_info.pack()

# Buat baton di tengah layar
center = Vector(WIDTH / 2, HEIGHT / 2)
baton = Baton(center, length=150)

# Mulai animasi
update()
window.mainloop()


In [50]:
import tkinter as tk
import math
from vector import Vector

# Tkinter setup
root = tk.Tk()
root.title("🎯 Arah Mover Menuju Target + Garis Bantu Segitiga")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Bola1 (posisi awal) dan Bola2 (target)
pos1 = Vector(100, 100)
pos2 = Vector(400, 300)

# Objek visual
radius = 10
ball = canvas.create_oval(0, 0, 0, 0, fill="red")
target = canvas.create_oval(pos2.x - 15, pos2.y - 15, pos2.x + 15, pos2.y + 15, outline="green")
line = canvas.create_line(0, 0, 0, 0, fill="blue", width=2)  # garis arah

# Garis bantu segitiga
line_alas = canvas.create_line(0, 0, 0, 0, fill="red", dash=(4, 2))
line_tinggi = canvas.create_line(0, 0, 0, 0, fill="blue", dash=(4, 2))
line_miring = canvas.create_line(0, 0, 0, 0, fill="gray", dash=(4, 2))

# Teks
text = canvas.create_text(10, 10, anchor="nw", font=("Arial", 11), fill="black", text="")

# Kecepatan
speed = 2

def update():
    global pos1

    # Hitung arah dari bola1 ke bola2
    direction = pos2.subbed(pos1)
    angle = math.atan2(direction.y, direction.x)  # Sudut arah (radian)
    direction_unit = direction.normalized()
    velocity = direction_unit.multed(speed)
    pos1 = Vector(pos1.x + velocity.x, pos1.y + velocity.y)

    # Update posisi bola & arah
    canvas.coords(ball, pos1.x - radius, pos1.y - radius, pos1.x + radius, pos1.y + radius)
    canvas.coords(line, pos1.x, pos1.y, pos1.x + direction_unit.x * 30, pos1.y + direction_unit.y * 30)

    # ===== Garis Bantu Segitiga =====
    # Alas: horizontal dari pos1 ke target.x (tetap di y = pos1.y)
    canvas.coords(line_alas, pos1.x, pos1.y, pos2.x, pos1.y)
    # Tinggi: vertical dari target.x, pos1.y ke pos2
    canvas.coords(line_tinggi, pos2.x, pos1.y, pos2.x, pos2.y)
    # Miring: miring dari target.x, pos1.y ke pos2
    canvas.coords(line_miring, pos1.x, pos1.y, pos2.x, pos2.y)
    # Info trigonometri
    delta_x = pos2.x - pos1.x
    delta_y = pos2.y - pos1.y

    info = f"""
📘 Trigonometri & Vector
Target: ({pos2.x}, {pos2.y})
Posisi Bola1: ({pos1.x:.1f}, {pos1.y:.1f})

Δx (alas : red) = {pos2.x} - {pos1.x:.1f} = {delta_x:.1f}
Δy (tinggi : blue) = {pos2.y} - {pos1.y:.1f} = {delta_y:.1f}

Sudut arah (radian) = atan2(Δy, Δx) = {angle:.2f}
Sudut arah (°)      = {math.degrees(angle):.1f}°

Arah unit vector:
= (cos(θ), sin(θ)) = ({math.cos(angle):.2f}, {math.sin(angle):.2f})
= {direction_unit}

Kecepatan: arah × {speed} = {velocity}
"""
    canvas.itemconfig(text, text=info)

    # Ulangi jika belum sampai
    if direction.magnitude() > speed:
        root.after(30, update)

update()
root.mainloop()


In [51]:
#4. Simulasi Mover Mengejar Target Bergerak
import tkinter as tk
import math
from vector import Vector

# Setup Tkinter
root = tk.Tk()
root.title("🎯 Simulasi Mover Mengejar Target Bergerak")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Bola1 (mover) dan bola2 (target)
pos1 = Vector(100, 100)
waypoints = [Vector(250, 300), Vector(250, 100), Vector(500, 150), Vector(100, 350), Vector(400, 350)]
waypoint_index = 0
pos2 = waypoints[waypoint_index]

# Visual
radius = 10
ball = canvas.create_oval(0, 0, 0, 0, fill="red")
target = canvas.create_oval(pos2.x - 10, pos2.y - 10, pos2.x + 10, pos2.y + 10, fill="green")
line = canvas.create_line(0, 0, 0, 0, fill="blue", width=2)
text = canvas.create_text(10, 10, anchor="nw", font=("Arial", 11), fill="black", text="")

# Kecepatan bola1
speed = 2

def update():
    global pos1, pos2, waypoint_index

    # Hitung arah dari bola1 ke bola2
    direction = pos2.subbed(pos1)
    distance = direction.magnitude()
    angle = math.atan2(direction.y, direction.x)
    direction_unit = direction.normalized()
    velocity = direction_unit.multed(speed)
    
    # Update posisi bola1
    if distance > speed:
        pos1 = Vector(pos1.x + velocity.x, pos1.y + velocity.y)
    else:
        # Jika bola1 sudah dekat, pindah ke waypoint berikutnya
        waypoint_index = (waypoint_index + 1) % len(waypoints)
        pos2 = waypoints[waypoint_index]
        canvas.coords(target, pos2.x - 10, pos2.y - 10, pos2.x + 10, pos2.y + 10)

    # Update tampilan bola dan garis
    canvas.coords(ball, pos1.x - radius, pos1.y - radius, pos1.x + radius, pos1.y + radius)
    canvas.coords(line, pos1.x, pos1.y, pos1.x + direction_unit.x * 30, pos1.y + direction_unit.y * 30)

    # Informasi perhitungan
    info = f"""
📘 Trigonometri & Vector Menuju Target
Waypoint ke-{waypoint_index + 1}: ({pos2.x}, {pos2.y})
Posisi Bola1: ({pos1.x:.1f}, {pos1.y:.1f})

Δx = {pos2.x} - {pos1.x:.1f} = {pos2.x - pos1.x:.1f}
Δy = {pos2.y} - {pos1.y:.1f} = {pos2.y - pos1.y:.1f}

Sudut arah (radian) = atan2(Δy, Δx) = {angle:.2f}
Sudut arah (°)      = {math.degrees(angle):.1f}°

Arah unit vector:
= (cos(θ), sin(θ)) = ({math.cos(angle):.2f}, {math.sin(angle):.2f})
= {direction_unit}

Kecepatan: arah × {speed} = {velocity}
"""
    canvas.itemconfig(text, text=info)

    # Ulangi setiap 30ms
    root.after(30, update)

update()
root.mainloop()


In [52]:
#SCRIPT 5 : # Simulasi mobil kotak yang bergerak mengikuti arah mouse
import tkinter as tk
import math
from vector import Vector

class Mover:
    def __init__(self, canvas, width, height):
        self.canvas = canvas  # tempat menggambar
        self.width = width    # lebar layar
        self.height = height  # tinggi layar
        self.position = Vector(width / 2, height / 2)  # posisi awal di tengah
        self.velocity = Vector(0, 0)  # kecepatan awal (diam)
        self.acceleration = Vector(0, 0)  # percepatan awal
        self.topspeed = 4  # kecepatan maksimum
        self.r = 16  # ukuran
        self.id = canvas.create_polygon(0, 0, 0, 0, 0, 0, fill="gray", outline="black")  # bentuk kotak

    def update(self, mouse_x, mouse_y):
        mouse = Vector(mouse_x, mouse_y)  # posisi mouse
        dir = mouse.subbed(self.position)    # arah menuju mouse
        dir.normalize()  # buat panjang vektor = 1
        dir.mult(0.5)    # perkecil pengaruhnya
        self.acceleration = dir  # set percepatan
        
        self.velocity.add(self.acceleration)  # tambah kecepatan
        self.velocity.limit(self.topspeed)    # batasi kecepatan maks
        self.position.add(self.velocity)      # update posisi

    def display(self):
        angle = self.velocity.heading()  # sudut berdasarkan arah gerak
        
        w, h = 30, 10  # ukuran kotak
        cx, cy = self.position.x, self.position.y  # posisi tengah
        
        cos_a = math.cos(angle)  # hitung komponen x
        sin_a = math.sin(angle)  # hitung komponen y
        
        points = []
        for dx, dy in [(-w/2, -h/2), (w/2, -h/2), (w/2, h/2), (-w/2, h/2)]:
            x = cx + dx * cos_a - dy * sin_a  # rotasi titik x
            y = cy + dx * sin_a + dy * cos_a  # rotasi titik y
            points.extend((x, y))
        
        self.canvas.coords(self.id, *points)  # update posisi kotak

    def check_edges(self):
        if self.position.x > self.width:  # jika keluar kanan
            self.position.x = 0           # muncul di kiri
        elif self.position.x < 0:         # jika keluar kiri
            self.position.x = self.width   # muncul di kanan
            
        if self.position.y > self.height:  # jika keluar bawah
            self.position.y = 0            # muncul di atas
        elif self.position.y < 0:          # jika keluar atas
            self.position.y = self.height  # muncul di bawah

# Setup Tkinter
root = tk.Tk()
root.title("Example 3.3: Pointing in the Direction of Motion")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

mover = Mover(canvas, WIDTH, HEIGHT)

def animate():
    canvas.delete("text")
    mouse_x = canvas.winfo_pointerx() - canvas.winfo_rootx()  # posisi mouse x
    mouse_y = canvas.winfo_pointery() - canvas.winfo_rooty()  # posisi mouse y

    mover.update(mouse_x, mouse_y)  # update posisi kotak
    mover.check_edges()             # cek tepi layar
    mover.display()                 # gambar ulang
    
    # Tampilkan teks posisi dan kecepatan
    canvas.create_text(10, 10, anchor="nw",
                       text=f"Pos: ({mover.position.x:.1f}, {mover.position.y:.1f})",
                       font=("Arial", 12), tag="text")
    canvas.create_text(10, 30, anchor="nw",
                       text=f"Vel: ({mover.velocity.x:.2f}, {mover.velocity.y:.2f})",
                       font=("Arial", 12), tag="text")

    root.after(30, animate)  # ulangi setiap 30ms

animate()
root.mainloop()


In [ ]:
import tkinter as tk
from vector import Vector

class Mover:
    def __init__(self, canvas, width, height):
        self.canvas = canvas
        self.width = width
        self.height = height
        self.position = Vector(width / 2, height / 2)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.topspeed = 5
        self.radius = 24
        self.shape = self.canvas.create_oval(0, 0, 0, 0, fill='lightblue', outline='black', width=2)
        # Info teks kosong
        self.mouse_pos = canvas.create_text(10, 10, anchor='nw', text="", font=('Arial', 10), fill='black')
        self.mover_pos = canvas.create_text(10, 30, anchor='nw', text="", font=('Arial', 10), fill='black')
        self.mover_dir = canvas.create_text(10, 50, anchor='nw', text="", font=('Arial', 10), fill='black')
        self.mover_nor = canvas.create_text(10, 70, anchor='nw', text="", font=('Arial', 10), fill='black')
        self.mover_velo = canvas.create_text(10, 90, anchor='nw', text="", font=('Arial', 10), fill='black')
        self.update()

    def update(self):
        # Dapatkan posisi mouse relatif ke canvas
        mouse_x = self.canvas.winfo_pointerx() - self.canvas.winfo_rootx()
        mouse_y = self.canvas.winfo_pointery() - self.canvas.winfo_rooty()
        mouse = Vector(mouse_x, mouse_y)
        # Update teks kosong dengan info posisi (x,y)
        self.canvas.itemconfig(self.mouse_pos, text=f"mouse: ({mouse})")
        self.canvas.itemconfig(self.mover_pos, text=f"Mover: ({self.position})")
        # Hitung arah, normalisasi, dan scaling
        dir = mouse.subbed(self.position)
        self.canvas.itemconfig(self.mover_dir, text=f"Arah: ({dir})")
        dir = dir.normalized().multed(0.2)
        self.canvas.itemconfig(self.mover_nor, text=f"Normalisi: ({dir})")
        
        # Percepatan dan kecepatan
        self.acceleration = dir
        self.velocity.add(self.acceleration)
        self.velocity.limit(self.topspeed)
        self.canvas.itemconfig(self.mover_velo, text=f"velocity: ({self.velocity})")
        self.position.add(self.velocity)

        # Update koordinat lingkaran
        x, y = self.position.x, self.position.y
        r = self.radius
        self.canvas.coords(self.shape, x - r, y - r, x + r, y + r)


        self.canvas.after(16, self.update)  # ~60 FPS



class App:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Mover Following Mouse - The Nature of Code")

        width, height = 640, 480
        self.canvas = tk.Canvas(self.root, width=width, height=height, bg='white')
        self.canvas.pack()

        self.mover = Mover(self.canvas, width, height)

        self.root.mainloop()

if __name__ == "__main__":
    App()


In [74]:
# SCRIPT 5 : Simulasi Konversi Koordinat Polar ke Kartesian
import tkinter as tk
import math

# === Setup Tkinter ===
root = tk.Tk()
root.title("Example 3.4: Polar to Cartesian")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Inisialisasi variabel polar ===
r = HEIGHT * 0.45     # Jari-jari (radius)
theta = 0             # Sudut awal

# Objek lingkaran dan garis
circle_id = canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black", width=2)
line_id = canvas.create_line(0, 0, 0, 0, fill="black", width=2)

def animate():
    global theta

    canvas.delete("text")
    canvas.configure(bg="white")

    # Konversi polar ke kartesian
    x = r * math.cos(theta)
    y = r * math.sin(theta)

    # Pusat koordinat di tengah canvas
    center_x = WIDTH / 2
    center_y = HEIGHT / 2

    # Koordinat akhir (x, y) ditranslasi ke posisi canvas
    px = center_x + x
    py = center_y + y

    # Update garis dari pusat ke titik
    canvas.coords(line_id, center_x, center_y, px, py)

    # Update lingkaran di posisi (px, py)
    radius = 24
    canvas.coords(circle_id, px - radius, py - radius, px + radius, py + radius)

    # Tampilkan data
    canvas.create_text(10, 10, anchor="nw", text=f"theta = {theta:.2f} rad", font=("Arial", 12), tag="text")
    canvas.create_text(10, 70, anchor="nw", text=f"theta = {math.degrees(theta):.2f} degree", font=("Arial", 12), tag="text")
    canvas.create_text(10, 30, anchor="nw", text=f"x = {x:.1f}", font=("Arial", 12), tag="text")
    canvas.create_text(10, 50, anchor="nw", text=f"y = {y:.1f}", font=("Arial", 12), tag="text")

    # Update sudut theta untuk rotasi
    theta += 0.02

    root.after(30, animate)

animate()
root.mainloop()


In [75]:
import tkinter as tk
import math

WIDTH, HEIGHT = 600, 600
CENTER_X, CENTER_Y = WIDTH // 2, HEIGHT // 2

root = tk.Tk()
root.title("Simulasi Polar ke Cartesian dengan Garis Bantu")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# ---------- Lingkaran utama ----------
circle_radius = 200
canvas.create_oval(CENTER_X - circle_radius, CENTER_Y - circle_radius,
                   CENTER_X + circle_radius, CENTER_Y + circle_radius,
                   outline="lightgray")

# ---------- Sliders ----------
theta_scale = tk.Scale(root, from_=0, to=360, label="Sudut θ (derajat)", orient="horizontal", length=300)
theta_scale.set(45)
theta_scale.pack()

radius_scale = tk.Scale(root, from_=0, to=200, label="Radius r", orient="horizontal", length=300)
radius_scale.set(100)
radius_scale.pack()

# ---------- Label hasil koordinat ----------
label_coords = tk.Label(root, text="", font=("Arial", 12))
label_coords.pack()

# ---------- Objek Canvas ----------
point = canvas.create_oval(0, 0, 0, 0, fill="red")
line_r = canvas.create_line(0, 0, 0, 0, fill="blue", width=2)         # garis radius r
line_x = canvas.create_line(0, 0, 0, 0, fill="gray", dash=(4, 2))     # proyeksi ke sumbu x
line_baseline = canvas.create_line(0, 0, 0, 0, fill="black", width=1) # sumbu x (θ = 0°)

# ---------- Label garis panjang ----------
text_r = canvas.create_text(0, 0, text="", fill="blue", font=("Arial", 10))
text_x = canvas.create_text(0, 0, text="", fill="black", font=("Arial", 10))
text_y = canvas.create_text(0, 0, text="", fill="black", font=("Arial", 10))

# ---------- Fungsi Update ----------
def update():
    theta_deg = theta_scale.get()
    r = radius_scale.get()
    theta_rad = math.radians(theta_deg)

    # Hitung posisi Cartesian
    x = r * math.cos(theta_rad)
    y = r * math.sin(theta_rad)
    screen_x = CENTER_X + x
    screen_y = CENTER_Y + y

    # Update titik
    r_dot = 6
    canvas.coords(point,
                  screen_x - r_dot, screen_y - r_dot,
                  screen_x + r_dot, screen_y + r_dot)

    # Garis radius (r)
    canvas.coords(line_r, CENTER_X, CENTER_Y, screen_x, screen_y)

    # Garis proyeksi Y (vertikal ke sumbu X)
    canvas.coords(line_x, screen_x, screen_y, screen_x, CENTER_Y)

    # Garis dasar sumbu X dari center ke sudut 0° (baseline)
    canvas.coords(line_baseline, CENTER_X, CENTER_Y, CENTER_X + r, CENTER_Y)

    # Update label
    label_coords.config(
        text=f"r = {r}, θ = {theta_deg}°\n→ x = {x:.2f}, y = {y:.2f}"
    )

    # Teks panjang garis-garis
    canvas.coords(text_r, (CENTER_X + screen_x) / 2, (CENTER_Y + screen_y) / 2)
    canvas.itemconfig(text_r, text=f"r = {r}")

    canvas.coords(text_y, screen_x + 10, (CENTER_Y + screen_y) / 2)
    canvas.itemconfig(text_y, text=f"y = {y:.2f}")

    canvas.coords(text_x, (CENTER_X + screen_x) / 2, CENTER_Y + 15)
    canvas.itemconfig(text_x, text=f"x = {x:.2f}")

    root.after(33, update)

update()
root.mainloop()


In [ ]:
# SCRIPT 7 : Simulasi jalur spiral menggunakan koordinat polar
import tkinter as tk
import math

# Setup Tkinter
root = tk.Tk()
root.title("Exercise 3.4: Spiral Path")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Inisialisasi polar ===
r = 0             # Radius mulai dari 0 (untuk spiral)
theta = 0         # Sudut awal
dot_radius = 5    # Ukuran titik spiral

# Simpan semua titik spiral
spiral_points = []

def animate():
    global theta, r

    # Konversi polar ke kartesian
    x = r * math.cos(theta)
    y = r * math.sin(theta)

    # Geser titik ke tengah canvas
    px = WIDTH / 2 + x
    py = HEIGHT / 2 + y

    # Simpan titik spiral
    spiral_points.append((px, py))

    # Gambar semua titik spiral
    for px, py in spiral_points:
        canvas.create_oval(px - dot_radius, py - dot_radius,
                           px + dot_radius, py + dot_radius,
                           fill="black", outline="")

    # Tambah sudut dan radius
    theta += 0.1   # makin besar = makin cepat berputar
    r += 0.3        # makin besar = spiral makin melebar

    root.after(30, animate)

animate()
root.mainloop()


In [ ]:
import tkinter as tk
import math
from vector import Vector

class Vehicle:
    def __init__(self, canvas, width, height):
        self.canvas = canvas
        self.width = width
        self.height = height

        self.position = Vector(width / 2, height / 2)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)

        self.damping = 0.995
        self.topspeed = 6
        self.heading = 0
        self.r = 16
        self.thrusting = False

        self.id = canvas.create_polygon(0, 0, 0, 0, 0, 0, fill="gray", outline="black")
        self.text_sudut = canvas.create_text(10, 30, anchor="nw", text="",font=("Arial", 12))
        self.text_velocity = canvas.create_text(10, 50, anchor="nw", text="",font=("Arial", 12))

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.mult(self.damping)
        self.velocity.limit(self.topspeed)
        self.canvas.itemconfig(self.text_velocity, text=f"Velocity: ({self.velocity})")
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)

    def applyForce(self, force):
        self.acceleration.add(force)

    def turn(self, angle):
        self.heading += angle

    def thrust(self):
        force = Vector.fromAngle(self.heading - math.pi / 2).multed(0.1)
        self.applyForce(force)
        self.thrusting = True

    def wrapEdges(self):
        buffer = self.r * 2
        if self.position.x > self.width + buffer:
            self.position.x = -buffer
        elif self.position.x < -buffer:
            self.position.x = self.width + buffer

        if self.position.y > self.height + buffer:
            self.position.y = -buffer
        elif self.position.y < -buffer:
            self.position.y = self.height + buffer

    def show(self):
        angle = self.heading
        self.canvas.itemconfig(self.text_sudut, text=f"Angle: ({math.degrees(angle) % 360})")
        cx, cy = self.position.x, self.position.y

        points = []
        front = Vector(0, -self.r * 1.5).rotated(angle).added(self.position)
        left = Vector(-self.r, self.r).rotated(angle).added(self.position)
        right = Vector(self.r, self.r).rotated(angle).added(self.position)

        points.extend([front.x, front.y, left.x, left.y, right.x, right.y])

        self.canvas.coords(self.id, *points)
        self.canvas.itemconfig(self.id, fill="red" if self.thrusting else "gray")
        self.thrusting = False

class App:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Vehicle Segitiga Sama Kaki - Steering & Wrap")

        WIDTH, HEIGHT = 640, 480
        self.canvas = tk.Canvas(self.root, width=WIDTH, height=HEIGHT, bg="white")
        self.canvas.pack()

        self.vehicle = Vehicle(self.canvas, WIDTH, HEIGHT)

        self.keys = {"left": False, "right": False, "up": False}

        self.root.bind("<KeyPress>", self.key_press)
        self.root.bind("<KeyRelease>", self.key_release)

        self.animate()
        self.root.mainloop()

    def key_press(self, event):
        if event.keysym == "Left":
            self.keys["left"] = True
        elif event.keysym == "Right":
            self.keys["right"] = True
        elif event.keysym == "Up":
            self.keys["up"] = True

    def key_release(self, event):
        if event.keysym == "Left":
            self.keys["left"] = False
        elif event.keysym == "Right":
            self.keys["right"] = False
        elif event.keysym == "Up":
            self.keys["up"] = False

    def animate(self):
        if self.keys["left"]:
            self.vehicle.turn(-0.05)
        if self.keys["right"]:
            self.vehicle.turn(0.05)
        if self.keys["up"]:
            self.vehicle.thrust()

        self.vehicle.update()
        self.vehicle.wrapEdges()
        self.vehicle.show()

        self.canvas.delete("info")
        self.canvas.create_text(10, 10, anchor="nw",
                                 text=f"Pos: ({self.vehicle.position.x:.1f}, {self.vehicle.position.y:.1f})",
                                 font=("Arial", 12), tag="info")

        self.root.after(30, self.animate)

if __name__ == "__main__":
    App()


In [89]:
# Simulasi Gerak Harmonik Vertikal + Grafik (COPY PASTE)
import tkinter as tk
import math
import time

# Setup awal
WIDTH, HEIGHT = 600, 400
CENTER_X, CENTER_Y = WIDTH // 2, HEIGHT // 2

root = tk.Tk()
root.title("Simulasi Gerak Harmonik Vertikal + Grafik")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Sliders
amp_slider = tk.Scale(root, from_=0, to=150, label="Amplitudo (px)", orient="horizontal", length=300)
amp_slider.set(100)
amp_slider.pack()

freq_slider = tk.Scale(root, from_=0.1, to=2.0, resolution=0.1, label="Frekuensi (Hz)", orient="horizontal", length=300)
freq_slider.set(0.5)
freq_slider.pack()

# Label rumus dan hasil
info_label = tk.Label(root, text="", font=("Arial", 12))
info_label.pack()

# Objek animasi
radius = 10
dot = canvas.create_oval(0, 0, 0, 0, fill="red")
line = canvas.create_line(CENTER_X, CENTER_Y, CENTER_X, CENTER_Y, fill="blue", width=2)
text_y = canvas.create_text(CENTER_X + 40, CENTER_Y, text="", font=("Arial", 10), fill="black")

# Grafik waktu vs simpangan
GRAPH_WIDTH = 400
GRAPH_HEIGHT = 150
GRAPH_X = 100
GRAPH_Y = HEIGHT - GRAPH_HEIGHT - 20
graph_points = []

# Garis bantu grafik
canvas.create_line(GRAPH_X, GRAPH_Y + GRAPH_HEIGHT//2, 
                  GRAPH_X + GRAPH_WIDTH, GRAPH_Y + GRAPH_HEIGHT//2, 
                  fill="gray", dash=(2, 2))  # Garis nol
canvas.create_text(GRAPH_X - 30, GRAPH_Y + GRAPH_HEIGHT//2, 
                  text="0", anchor="e", font=("Arial", 8))
canvas.create_text(GRAPH_X - 30, GRAPH_Y, 
                  text=f"{amp_slider.get()}", anchor="e", font=("Arial", 8))
canvas.create_text(GRAPH_X - 30, GRAPH_Y + GRAPH_HEIGHT, 
                  text=f"-{amp_slider.get()}", anchor="e", font=("Arial", 8))

start_time = time.time()

def update():
    global graph_points
    
    now = time.time()
    t = now - start_time  # waktu dalam detik

    A = amp_slider.get()
    f = freq_slider.get()
    omega = 2 * math.pi * f
    y_offset = A * math.sin(omega * t)

    # Update posisi titik
    y = CENTER_Y + y_offset
    canvas.coords(dot, CENTER_X - radius, y - radius, CENTER_X + radius, y + radius)
    canvas.coords(line, CENTER_X, CENTER_Y, CENTER_X, y)
    canvas.coords(text_y, CENTER_X + 40, y)
    canvas.itemconfig(text_y, text=f"y(t) = {y_offset:.1f}px")

    # Update grafik
    graph_points.append((t, y_offset))
    if len(graph_points) > 100:  # Batasi jumlah titik yang disimpan
        graph_points.pop(0)
    
    canvas.delete("graph")  # Hapus grafik sebelumnya
    
    # Gambar grafik baru
    for i in range(1, len(graph_points)):
        x1 = GRAPH_X + (i-1) * (GRAPH_WIDTH / 100)
        y1 = GRAPH_Y + GRAPH_HEIGHT//2 - (graph_points[i-1][1] * (GRAPH_HEIGHT/2/A))
        x2 = GRAPH_X + i * (GRAPH_WIDTH / 100)
        y2 = GRAPH_Y + GRAPH_HEIGHT//2 - (graph_points[i][1] * (GRAPH_HEIGHT/2/A))
        canvas.create_line(x1, y1, x2, y2, fill="green", width=2, tag="graph")
    
    # Update label skala grafik
    canvas.itemconfig(canvas.find_withtag("amplitude_text"), text=f"{A}")
    canvas.itemconfig(canvas.find_withtag("negative_amplitude_text"), text=f"-{A}")

    # Tampilkan rumus dan nilai
    info_label.config(
        text=f"Rumus: y(t) = A × sin(2πft)\nA = {A} px, f = {f} Hz, t = {t:.2f} s, y = {y_offset:.2f} px\n"
             f"Frekuensi sudut (ω) = {omega:.2f} rad/s"
    )

    root.after(20, update)

update()
root.mainloop()


In [91]:
# Simulasi Gerak Harmonik Horizontal (Sumbu X)
import tkinter as tk
import math
import time

# Ukuran canvas dan titik pusat
WIDTH, HEIGHT = 600, 400
CENTER_X, CENTER_Y = WIDTH // 2, HEIGHT // 2

root = tk.Tk()
root.title("Simulasi Gerak Harmonik Horizontal (Sumbu X)")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Sliders
amp_slider = tk.Scale(root, from_=0, to=200, label="Amplitudo (px)", orient="horizontal", length=300)
amp_slider.set(100)
amp_slider.pack()

freq_slider = tk.Scale(root, from_=0.1, to=2.0, resolution=0.1, label="Frekuensi (Hz)", orient="horizontal", length=300)
freq_slider.set(0.5)
freq_slider.pack()

# Label rumus dan hasil
info_label = tk.Label(root, text="", font=("Arial", 12))
info_label.pack()

# Objek animasi
radius = 10
dot = canvas.create_oval(0, 0, 0, 0, fill="red")
line = canvas.create_line(CENTER_X, CENTER_Y, CENTER_X, CENTER_Y, fill="blue", width=2)
text_x = canvas.create_text(CENTER_X, CENTER_Y - 30, text="", font=("Arial", 10), fill="black")

start_time = time.time()

# Fungsi update animasi
def update():
    now = time.time()
    t = now - start_time  # waktu dalam detik

    A = amp_slider.get()
    f = freq_slider.get()
    omega = 2 * math.pi * f
    x_offset = A * math.sin(omega * t)

    # Posisi titik
    x = CENTER_X + x_offset
    canvas.coords(dot, x - radius, CENTER_Y - radius, x + radius, CENTER_Y + radius)
    canvas.coords(line, CENTER_X, CENTER_Y, x, CENTER_Y)
    canvas.coords(text_x, x, CENTER_Y - 30)
    canvas.itemconfig(text_x, text=f"x(t) = {x_offset:.1f}px")

    # Update label informasi
    info_label.config(
        text=f"Rumus: x(t) = A × sin(2πft)\nA = {A}, f = {f} Hz, t = {t:.2f}s, x = {x_offset:.2f}px"
    )

    root.after(20, update)

update()
root.mainloop()


In [93]:
# SCRIPT 10 : Simulasi Gerakan Osilator
import tkinter as tk
import math
import random
from vector import Vector

# === Konstanta Layar ===
WIDTH, HEIGHT = 640, 240

# === Kelas Oscillator ===
class Oscillator:
    def __init__(self, canvas):
        self.canvas = canvas
        self.angle = Vector(0, 0)
        self.angle_velocity = Vector(random.uniform(-0.05, 0.05), random.uniform(-0.05, 0.05))
        self.amplitude = Vector(random.uniform(20, WIDTH / 2), random.uniform(20, HEIGHT / 2))

        # Buat objek visual (garis dan lingkaran)
        self.line_id = canvas.create_line(0, 0, 0, 0, fill="black", width=2)
        self.circle_id = canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black")

    def update(self):
        self.angle.add(self.angle_velocity)

    def display(self):
        # Konversi sudut sinusoidal ke posisi (x, y)
        x = math.sin(self.angle.x) * self.amplitude.x
        y = math.sin(self.angle.y) * self.amplitude.y

        center_x, center_y = WIDTH / 2, HEIGHT / 2
        px, py = center_x + x, center_y + y

        # Update garis dari pusat ke titik osilasi
        self.canvas.coords(self.line_id, center_x, center_y, px, py)

        # Update lingkaran
        radius = 16
        self.canvas.coords(
            self.circle_id,
            px - radius, py - radius,
            px + radius, py + radius
        )

# === Setup Tkinter ===
root = tk.Tk()
root.title("Example 3.7: Oscillator Objects")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Buat banyak oscillator ===
oscillators = [Oscillator(canvas) for _ in range(10)]

# === Fungsi animasi utama ===
def animate():
    for osc in oscillators:
        osc.update()
        osc.display()
    root.after(30, animate)

animate()
root.mainloop()


In [94]:
# SCRIPT 11 : Simulasi gelombang statis menggunakan sinus
import tkinter as tk
import math

# === Konstanta Layar ===
WIDTH, HEIGHT = 640, 240
ANGLE_VELOCITY = 0.2
AMPLITUDE = 100
SPACING = 24
RADIUS = 24

# === Setup Tkinter ===
root = tk.Tk()
root.title("Example 3.8: Static Wave")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Gambar lingkaran gelombang ===
angle = 0
for x in range(0, WIDTH + 1, SPACING):
    # 1) Hitung posisi y menggunakan sinus
    y = AMPLITUDE * math.sin(angle)
    center_y = y + HEIGHT / 2

    # 2) Gambar lingkaran di posisi (x, center_y)
    canvas.create_oval(
        x - RADIUS, center_y - RADIUS,
        x + RADIUS, center_y + RADIUS,
        fill="gray", outline="black", width=2
    )

    # 3) Tambahkan sudut
    angle += ANGLE_VELOCITY

root.mainloop()


In [95]:
import tkinter as tk
import math
from vector import Vector

# Konfigurasi jendela dan kanvas
WIDTH, HEIGHT = 640, 240
STEP = 24         # Jarak antar titik gelombang
CIRCLE_SIZE = 48  # Diameter lingkaran

class WaveSimulation:
    def __init__(self, root):
        self.canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
        self.canvas.pack()

        self.start_angle = 0.0       # Mirip variabel startAngle di Processing
        self.angle_velocity = 0.2    # Kecepatan perubahan sudut antar titik

        self.circles = []  # List untuk menyimpan ID lingkaran di canvas

        # Inisialisasi lingkaran pada posisi awal
        for x in range(0, WIDTH + 1, STEP):
            y = HEIGHT // 2  # Posisi y sementara
            circle_id = self.canvas.create_oval(
                x - CIRCLE_SIZE//2, y - CIRCLE_SIZE//2,
                x + CIRCLE_SIZE//2, y + CIRCLE_SIZE//2,
                fill="gray", outline="black"
            )
            self.circles.append(circle_id)

        # Mulai animasi
        self.animate()

    def animate(self):
        # Bersihkan background dengan cara mengganti warna semua oval
        self.canvas.delete("all")

        angle = self.start_angle
        self.start_angle += 0.02  # Seiring waktu sudut awal bertambah

        # Gambar ulang semua lingkaran mengikuti pola gelombang
        for i, x in enumerate(range(0, WIDTH + 1, STEP)):
            # Ubah nilai sin menjadi nilai y (0 - HEIGHT)
            sin_value = math.sin(angle)
            y = self.map_value(sin_value, -1, 1, 0, HEIGHT)

            # Gambar ulang lingkaran
            circle_id = self.canvas.create_oval(
                x - CIRCLE_SIZE//2, y - CIRCLE_SIZE//2,
                x + CIRCLE_SIZE//2, y + CIRCLE_SIZE//2,
                fill="gray", outline="black"
            )

            angle += self.angle_velocity

        # Ulangi fungsi animate setiap 30 milidetik (~33 FPS)
        self.canvas.after(30, self.animate)

    @staticmethod
    def map_value(value, start1, stop1, start2, stop2):
        # Fungsi seperti map() di Processing: ubah nilai dari satu rentang ke rentang lain
        return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Gelombang Sine dengan Tkinter")
    sim = WaveSimulation(root)
    root.mainloop()


In [97]:
import tkinter as tk
from vector import Vector
from perlin_noise import PerlinNoise

# Konfigurasi jendela
WIDTH, HEIGHT = 640, 240
STEP = 24
CIRCLE_SIZE = 48

class WaveWithNoise:
    def __init__(self, root):
        self.canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
        self.canvas.pack()

        # Inisialisasi noise generator dan offset
        self.noise = PerlinNoise(octaves=1)
        self.x_offset_start = 0.0   # Nilai awal untuk offset
        self.offset_increment = 0.1 # Seberapa cepat noise bergerak

        self.circle_ids = []  # Menyimpan ID lingkaran

        for x in range(0, WIDTH + 1, STEP):
            y = HEIGHT // 2
            circle_id = self.canvas.create_oval(
                x - CIRCLE_SIZE//2, y - CIRCLE_SIZE//2,
                x + CIRCLE_SIZE//2, y + CIRCLE_SIZE//2,
                fill="lightblue", outline="black"
            )
            self.circle_ids.append(circle_id)

        self.animate()

    def animate(self):
        # Bersihkan canvas
        self.canvas.delete("all")

        x_offset = self.x_offset_start
        self.x_offset_start += 0.01  # Perubahan waktu global (seperti frameCount)

        for i, x in enumerate(range(0, WIDTH + 1, STEP)):
            # Ambil nilai noise untuk x_offset
            noise_value = self.noise(x_offset)

            # Mapping noise (-1 to 1) menjadi (0 to HEIGHT)
            amplitude = HEIGHT / 3  # tinggi gelombang
            center_y = HEIGHT / 2   # titik tengah vertikal
            y = self.map_value(noise_value, 0, 1, center_y + amplitude, center_y - amplitude)

            # Gambar lingkaran
            circle_id = self.canvas.create_oval(
                x - CIRCLE_SIZE//2, y - CIRCLE_SIZE//2,
                x + CIRCLE_SIZE//2, y + CIRCLE_SIZE//2,
                fill="lightblue", outline="black"
            )

            self.circle_ids[i] = circle_id
            x_offset += self.offset_increment  # Tambah offset untuk gelombang menyebar

        self.canvas.after(30, self.animate)

    @staticmethod
    def map_value(value, start1, stop1, start2, stop2):
        return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Perlin Noise Wave - Tkinter")
    app = WaveWithNoise(root)
    root.mainloop()


In [98]:
from tkinter import *
import math

from vector import Vector

# --- Konstanta global ---
WIDTH, HEIGHT = 640, 240
CIRCLE_SIZE = 24

# --- Kelas Wave ---
class Wave:
    def __init__(self, canvas, x, y, w, amplitude, period):
        self.canvas = canvas
        self.xspacing = 8
        self.w = w
        self.origin = Vector(x, y)
        self.theta = 0.0
        self.amplitude = amplitude
        self.period = period
        self.dx = (math.tau / self.period) * self.xspacing
        self.yvalues = [0.0 for _ in range(int(self.w / self.xspacing))]
        self.circle_ids = [None for _ in range(len(self.yvalues))]

    def update(self):
        self.theta += 0.02
        x = self.theta
        for i in range(len(self.yvalues)):
            self.yvalues[i] = math.sin(x) * self.amplitude
            x += self.dx

    def show(self):
        for i in range(len(self.yvalues)):
            x_pos = self.origin.x + i * self.xspacing
            y_pos = self.origin.y + self.yvalues[i]
            if self.circle_ids[i] is None:
                self.circle_ids[i] = self.canvas.create_oval(
                    x_pos - CIRCLE_SIZE//2, y_pos - CIRCLE_SIZE//2,
                    x_pos + CIRCLE_SIZE//2, y_pos + CIRCLE_SIZE//2,
                    fill="lightblue", outline="black"
                )
            else:
                self.canvas.coords(
                    self.circle_ids[i],
                    x_pos - CIRCLE_SIZE//2, y_pos - CIRCLE_SIZE//2,
                    x_pos + CIRCLE_SIZE//2, y_pos + CIRCLE_SIZE//2
                )

# --- Aplikasi utama ---
class App:
    def __init__(self, root):
        self.canvas = Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
        self.canvas.pack()

        self.wave0 = Wave(self.canvas, 50, 75, 100, 20, 600)
        self.wave1 = Wave(self.canvas, 300, 120, 300, 40, 180)

        self.animate()

    def animate(self):
        self.wave0.update()
        self.wave0.show()

        self.wave1.update()
        self.wave1.show()

        self.canvas.after(30, self.animate)

# --- Jalankan ---
if __name__ == "__main__":
    root = Tk()
    root.title("Exercise 3.11 - OOP Wave")
    app = App(root)
    root.mainloop()


In [102]:
import tkinter as tk
import math
from vector import Vector

# Kelas Pendulum
class Pendulum:
    def __init__(self, origin_x, origin_y, r, canvas):
        self.origin = Vector(origin_x, origin_y)
        self.r = r
        self.angle = math.pi / 4  # Sudut awal 45 derajat
        self.a_velocity = 0.0     # Kecepatan sudut
        self.a_acceleration = 0.0 # Percepatan sudut
        self.damping = 0.995      # Redaman
        self.ball_radius = 24
        self.dragging = False
        self.canvas = canvas

        # Posisi bob
        self.bob = Vector()
        self.update_bob()

        # Objek canvas
        self.line = canvas.create_line(0, 0, 0, 0, width=2)
        self.circle = canvas.create_oval(0, 0, 0, 0, fill='gray')
        self.text = canvas.create_text(10, 10, anchor='nw', font=("Arial", 12), fill="black")

    def update_bob(self):
        self.bob.set(self.r * math.sin(self.angle), self.r * math.cos(self.angle))
        self.bob.add(self.origin)

    def update(self):
        gravity = 0.4
        self.a_acceleration = (-gravity / self.r) * math.sin(self.angle)
        self.a_velocity += self.a_acceleration
        self.angle += self.a_velocity
        self.a_velocity *= self.damping
        self.update_bob()

    def show(self):
        self.canvas.coords(self.line, self.origin.x, self.origin.y, self.bob.x, self.bob.y)
        self.canvas.coords(self.circle,
                           self.bob.x - self.ball_radius, self.bob.y - self.ball_radius,
                           self.bob.x + self.ball_radius, self.bob.y + self.ball_radius)
        info = (
            f"Sudut: {math.degrees(self.angle):.2f}°\n"
            f"Percepatan: {self.a_acceleration:.4f} rad/s²\n"
            f"Kecepatan: {self.a_velocity:.4f} rad/s"
        )
        self.canvas.itemconfig(self.text, text=info)

# Tkinter setup
root = tk.Tk()
root.title("Simulasi Pendulum")

canvas = tk.Canvas(root, width=640, height=360, bg='white')
canvas.pack()

pendulum = Pendulum(320, 50, 175, canvas)

# Loop animasi
def update():
    pendulum.update()
    pendulum.show()
    root.after(16, update)

update()
root.mainloop()


In [103]:
import tkinter as tk
import math
from vector import Vector

# Kelas Pendulum
class Pendulum:
    def __init__(self, origin_x, origin_y, r, canvas):
        self.origin = Vector(origin_x, origin_y)
        self.r = r
        self.angle = math.pi / 4  # Sudut awal 45 derajat
        self.a_velocity = 0.0     # Kecepatan sudut
        self.a_acceleration = 0.0 # Percepatan sudut
        self.damping = 0.995      # Redaman
        self.ball_radius = 24
        self.dragging = False
        self.canvas = canvas

        # Posisi bob
        self.bob = Vector()
        self.update_bob()

        # Objek canvas
        self.line = canvas.create_line(0, 0, 0, 0, width=2)
        self.circle = canvas.create_oval(0, 0, 0, 0, fill='gray')
        self.text = canvas.create_text(10, 10, anchor='nw', font=("Arial", 12), fill="black")

    def update_bob(self):
        self.bob.set(self.r * math.sin(self.angle), self.r * math.cos(self.angle))
        self.bob.add(self.origin)

    def update(self):
        if not self.dragging:
            gravity = 0.4
            self.a_acceleration = (-gravity / self.r) * math.sin(self.angle)
            self.a_velocity += self.a_acceleration
            self.angle += self.a_velocity
            self.a_velocity *= self.damping
        self.update_bob()

    def show(self):
        self.canvas.coords(self.line, self.origin.x, self.origin.y, self.bob.x, self.bob.y)
        self.canvas.coords(self.circle,
                           self.bob.x - self.ball_radius, self.bob.y - self.ball_radius,
                           self.bob.x + self.ball_radius, self.bob.y + self.ball_radius)
        info = (
            f"Sudut: {math.degrees(self.angle):.2f}°\n"
            f"Percepatan: {self.a_acceleration:.4f} rad/s²\n"
            f"Kecepatan: {self.a_velocity:.4f} rad/s"
        )
        self.canvas.itemconfig(self.text, text=info)

    def clicked(self, mx, my):
        d = math.hypot(mx - self.bob.x, my - self.bob.y)
        if d < self.ball_radius:
            self.dragging = True

    def stop_dragging(self):
        self.a_velocity = 0
        self.dragging = False

    def drag(self, mx, my):
        if self.dragging:
            dx = mx - self.origin.x
            dy = my - self.origin.y
            self.angle = math.atan2(dx, dy)  # Sudut dari sumbu vertikal, dibalik supaya arah mouse benar

# Tkinter setup
root = tk.Tk()
root.title("Simulasi Pendulum")

canvas = tk.Canvas(root, width=640, height=360, bg='white')
canvas.pack()

pendulum = Pendulum(320, 50, 175, canvas)

# Loop animasi
def update():
    pendulum.update()
    pendulum.show()
    root.after(16, update)

# Mouse events
def on_mouse_press(event):
    pendulum.clicked(event.x, event.y)

def on_mouse_release(event):
    pendulum.stop_dragging()

def on_mouse_drag(event):
    pendulum.drag(event.x, event.y)

# Bind mouse
canvas.bind("<ButtonPress-1>", on_mouse_press)
canvas.bind("<B1-Motion>", on_mouse_drag)
canvas.bind("<ButtonRelease-1>", on_mouse_release)

update()
root.mainloop()


In [104]:
import tkinter as tk
from vector import Vector

# ==== Bob (bola) ====
class Bob:
    def __init__(self, x, y, canvas):
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.mass = 20
        self.radius = 20
        self.dragging = False
        self.canvas = canvas
        self.id = canvas.create_oval(x - self.radius, y - self.radius,
                                     x + self.radius, y + self.radius,
                                     fill="blue")

    def apply_force(self, force):
        # F = m * a → a = F / m
        f = force.dived(self.mass)
        self.acceleration = self.acceleration.added(f)

    def update(self):
        if not self.dragging:
            # v = v + a
            self.velocity = self.velocity.added(self.acceleration)
            # s = s + v
            self.position = self.position.added(self.velocity)
        self.acceleration = Vector(0, 0)
        self.canvas.coords(self.id,
            self.position.x - self.radius,
            self.position.y - self.radius,
            self.position.x + self.radius,
            self.position.y + self.radius)

    def handle_click(self, mx, my):
        d = Vector(mx, my).subbed(self.position).mag()
        if d < self.radius:
            self.dragging = True

    def stop_dragging(self):
        self.dragging = False

    def handle_drag(self, mx, my):
        if self.dragging:
            self.position = Vector(mx, my)
            self.velocity = Vector(0, 0)

# ==== Spring ====
class Spring:
    def __init__(self, x, y, rest_length, canvas):
        self.anchor = Vector(x, y)
        self.rest_length = rest_length
        self.k = 0.2  # konstanta pegas
        self.canvas = canvas
        self.line_id = canvas.create_line(x, y, x, y)
        self.anchor_id = canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill="black")

    def connect(self, bob):
        # hitung gaya pegas
        force = bob.position.subbed(self.anchor)  # arah gaya: dari anchor ke bob
        d = force.mag()  # panjang saat ini
        stretch = d - self.rest_length  # selisih panjang

        # Fspring = -k * x
        force = force.normalized().multed(-self.k * stretch)
        bob.apply_force(force)
        return force  # dikembalikan agar bisa ditampilkan

    def constrain_length(self, bob, minlen, maxlen):
        direction = bob.position.subbed(self.anchor)
        d = direction.mag()
        if d < minlen:
            direction = direction.normalized().multed(minlen)
            bob.position = self.anchor.added(direction)
            bob.velocity = Vector(0, 0)
        elif d > maxlen:
            direction = direction.normalized().multed(maxlen)
            bob.position = self.anchor.added(direction)
            bob.velocity = Vector(0, 0)

    def show_line(self, bob):
        self.canvas.coords(self.line_id,
                           bob.position.x, bob.position.y,
                           self.anchor.x, self.anchor.y)

# ==== Main App ====
def main():
    root = tk.Tk()
    root.title("Spring Simulation")
    canvas = tk.Canvas(root, width=640, height=300, bg="white")
    canvas.pack()

    spring = Spring(320, 10, 100, canvas)
    bob = Bob(320, 100, canvas)

    text_id_force = canvas.create_text(10, 260, anchor="w", text="", font=("Consolas", 12))
    text_id_velocity = canvas.create_text(10, 280, anchor="w", text="", font=("Consolas", 12))
    text_id_accel = canvas.create_text(300, 280, anchor="w", text="", font=("Consolas", 12))

    mouse = {"x": 0, "y": 0}

    def update():
        # gaya gravitasi konstan ke bawah
        gravity = Vector(0, 0.98)
        bob.apply_force(gravity)

        bob.update()
        bob.handle_drag(mouse["x"], mouse["y"])

        # hitung dan aplikasikan gaya pegas ke bob
        spring_force = spring.connect(bob)

        # batasi panjang pegas
        spring.constrain_length(bob, 30, 200)

        # update garis pegas
        spring.show_line(bob)

        # tampilkan gaya, kecepatan, percepatan
        canvas.itemconfig(text_id_force,
                          text=f"F_spring = ({spring_force.x:.2f}, {spring_force.y:.2f})")
        canvas.itemconfig(text_id_velocity,
                          text=f"velocity  = ({bob.velocity.x:.2f}, {bob.velocity.y:.2f})")
        canvas.itemconfig(text_id_accel,
                          text=f"accel     = ({bob.acceleration.x:.2f}, {bob.acceleration.y:.2f})")

        root.after(16, update)  # ~60 FPS

    def on_mouse_press(event):
        bob.handle_click(event.x, event.y)

    def on_mouse_release(event):
        bob.stop_dragging()

    def on_mouse_motion(event):
        mouse["x"] = event.x
        mouse["y"] = event.y

    canvas.bind("<ButtonPress-1>", on_mouse_press)
    canvas.bind("<ButtonRelease-1>", on_mouse_release)
    canvas.bind("<Motion>", on_mouse_motion)

    update()
    root.mainloop()

if __name__ == "__main__":
    main()


In [106]:
from tkinter import *
from vector import Vector
import math

class Bob:
    def __init__(self, x, y, canvas):
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.mass = 24
        self.damping = 0.98
        self.dragging = False
        self.drag_offset = Vector(0, 0)
        self.canvas = canvas
        self.id = canvas.create_oval(x - self.mass / 2, y - self.mass / 2,
                                     x + self.mass / 2, y + self.mass / 2,
                                     fill="gray")

    def apply_force(self, force):
        f = force.dived(self.mass)
        self.acceleration = self.acceleration.added(f)

    def update(self):
        if not self.dragging:
            self.velocity = self.velocity.added(self.acceleration)
            self.velocity = self.velocity.multed(self.damping)
            self.position = self.position.added(self.velocity)
            self.acceleration = Vector(0, 0)

        r = self.mass / 2
        self.canvas.coords(self.id,
                           self.position.x - r, self.position.y - r,
                           self.position.x + r, self.position.y + r)

    def handle_click(self, mx, my):
        d = math.dist([mx, my], [self.position.x, self.position.y])
        if d < self.mass:
            self.dragging = True
            self.drag_offset = Vector(self.position.x - mx, self.position.y - my)

    def stop_dragging(self):
        self.dragging = False

    def handle_drag(self, mx, my):
        if self.dragging:
            self.position = Vector(mx, my).added(self.drag_offset)

class Spring:
    def __init__(self, a, b, length, canvas):
        self.a = a
        self.b = b
        self.length = length
        self.k = 0.2
        self.canvas = canvas
        self.id = canvas.create_line(a.position.x, a.position.y,
                                     b.position.x, b.position.y,
                                     fill="black", width=2)

    def update(self):
        force = self.a.position.subbed(self.b.position)
        stretch = force.mag() - self.length
        force = force.normalized().multed(-1 * self.k * stretch)
        self.a.apply_force(force)
        self.b.apply_force(force.multed(-1))
        self.canvas.coords(self.id,
                           self.a.position.x, self.a.position.y,
                           self.b.position.x, self.b.position.y)

def main():
    root = Tk()
    root.title("Spring Array Simulation")
    canvas = Canvas(root, width=640, height=360, bg="white")
    canvas.pack()

    bobs = [Bob(320, i * 40 + 50, canvas) for i in range(5)]
    springs = [Spring(bobs[i], bobs[i+1], 40, canvas) for i in range(4)]
    mouse = {"x": 0, "y": 0}

    def on_mouse_press(event):
        for b in bobs:
            b.handle_click(event.x, event.y)

    def on_mouse_release(event):
        for b in bobs:
            b.stop_dragging()

    def on_mouse_motion(event):
        mouse["x"], mouse["y"] = event.x, event.y

    def update():
        for b in bobs:
            b.handle_drag(mouse["x"], mouse["y"])
        for s in springs:
            s.update()
        for b in bobs:
            b.update()
        canvas.after(16, update)

    canvas.bind("<ButtonPress-1>", on_mouse_press)
    canvas.bind("<ButtonRelease-1>", on_mouse_release)
    canvas.bind("<Motion>", on_mouse_motion)

    update()
    root.mainloop()

if __name__ == "__main__":
    main()


In [105]:
import tkinter as tk
import math
import random
from vector import Vector

class Oscillator:
    def __init__(self, canvas, amplitude):
        self.canvas = canvas
        self.theta = 0
        self.amplitude = amplitude
        self.line = canvas.create_line(0, 0, 0, 0, fill='black')
        self.dot = canvas.create_oval(0, 0, 0, 0, fill='black')

    def update(self, vel_mag):
        self.theta += vel_mag

    def display(self, pos):
        x = math.cos(self.theta) * self.amplitude
        self.canvas.coords(self.line, pos.x, pos.y, pos.x + x, pos.y)
        r = 4
        self.canvas.coords(self.dot, pos.x + x - r, pos.y - r, pos.x + x + r, pos.y + r)

class Crawler:
    def __init__(self, canvas, width, height):
        self.canvas = canvas
        self.pos = Vector(random.uniform(0, width), random.uniform(0, height))
        self.vel = Vector(random.uniform(-1, 1), random.uniform(-1, 1))
        self.acc = Vector()
        self.mass = random.uniform(8, 16)
        self.radius = self.mass
        self.id = canvas.create_oval(0, 0, 0, 0, fill='gray')
        self.osc = Oscillator(canvas, self.mass * 2)

    def apply_force(self, force):
        f = force.dived(self.mass)
        self.acc = self.acc.added(f)

    def update(self):
        self.vel = self.vel.added(self.acc)
        self.pos = self.pos.added(self.vel)
        self.acc = Vector()
        self.osc.update(self.vel.mag() / 10)

    def display(self):
        r = self.radius
        self.canvas.coords(self.id, self.pos.x - r, self.pos.y - r, self.pos.x + r, self.pos.y + r)
        self.osc.display(self.pos)

class Attractor:
    def __init__(self, canvas, pos, mass, g):
        self.canvas = canvas
        self.pos = pos
        self.mass = mass
        self.G = g
        self.radius = mass
        self.dragging = False
        self.rollover = False
        self.drag_offset = Vector()
        self.id = canvas.create_oval(0, 0, 0, 0, fill='lightblue')

    def attract(self, crawler):
        force = self.pos.subbed(crawler.pos)
        d = max(5.0, min(25.0, force.mag()))
        force = force.normalized()
        strength = (self.G * self.mass * crawler.mass) / (d * d)
        return force.multed(strength)

    def render(self):
        fill = 'gray' if self.dragging else 'lightgreen' if self.rollover else 'lightblue'
        self.canvas.itemconfig(self.id, fill=fill)
        r = self.radius
        self.canvas.coords(self.id, self.pos.x - r, self.pos.y - r, self.pos.x + r, self.pos.y + r)

    def clicked(self, mx, my):
        d = math.hypot(mx - self.pos.x, my - self.pos.y)
        if d < self.radius:
            self.dragging = True
            self.drag_offset = Vector(self.pos.x - mx, self.pos.y - my)

    def stop_dragging(self):
        self.dragging = False

    def drag(self, mx, my):
        if self.dragging:
            self.pos.x = mx + self.drag_offset.x
            self.pos.y = my + self.drag_offset.y

    def rollover_check(self, mx, my):
        d = math.hypot(mx - self.pos.x, my - self.pos.y)
        self.rollover = d < self.radius

    def go(self):
        self.render()

# --- Main Application ---
def main():
    root = tk.Tk()
    root.title("Attraction Array with Oscillation")
    width, height = 640, 360
    canvas = tk.Canvas(root, width=width, height=height, bg='white')
    canvas.pack()

    attractor = Attractor(canvas, Vector(width / 2, height / 2), 20, 0.4)
    crawlers = [Crawler(canvas, width, height) for _ in range(6)]

    mouse = {'x': 0, 'y': 0}

    def draw():
        attractor.rollover_check(mouse['x'], mouse['y'])
        attractor.drag(mouse['x'], mouse['y'])
        attractor.go()

        for c in crawlers:
            f = attractor.attract(c)
            c.apply_force(f)
            c.update()
            c.display()

        root.after(16, draw)

    def on_mouse_move(event):
        mouse['x'], mouse['y'] = event.x, event.y

    def on_mouse_down(event):
        attractor.clicked(event.x, event.y)

    def on_mouse_up(event):
        attractor.stop_dragging()

    canvas.bind("<Motion>", on_mouse_move)
    canvas.bind("<ButtonPress-1>", on_mouse_down)
    canvas.bind("<ButtonRelease-1>", on_mouse_up)

    draw()
    root.mainloop()

if __name__ == "__main__":
    main()
